**Demo for `teneva.core.tensor`**

---

This module contains the basic operations and utilities for TT-tensors, including "add", "mul", "norm", "mean", etc.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `accuracy`

Compute $\frac{|| Y_1 - Y_2 ||}{|| Y_2 ||}$ for tensors in the TT-format.

In [2]:
Y1 = teneva.rand([5]*10, 2)                 # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.add(Y1, teneva.mul(1.E-4, Y1))  # The TT-tensor Y1 + eps * Y1 (eps = 1.E-4)
eps = teneva.accuracy(Y1, Y2)               # The relative difference ("accuracy")
print(f'Accuracy     : {eps:-8.2e}')

Accuracy     : 1.00e-04


## Function `add`

Compute element wise sum $Y = Y_1 + Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [3]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [4]:
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 3 = 5)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  5  5  5  5  5  5  5  5  1 



In [5]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full + Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 1.08e-16


This function also supports float argument:

In [6]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [7]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [8]:
Y1_full = 42.                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)           # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full + Y2_full

e = np.linalg.norm(Y_full - Z_full) # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)

print(f'Error     : {e:-8.2e}')     # Rel. error for TT-tensor vs full tensor

Error     : 9.12e-17


If both arguments are numbers, then function returns the sum of numbers:

In [9]:
Y1 = 40.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.add(Y1, Y2)      # Compute the sum of Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `add_many`

Compute element wise sum $Y = Y_1 + Y_2 + \ldots + Y_m$ for the given TT-tensors given as lists of TT-cores. The result is truncated to the given accuracy `e` and/or maximum TT-rank `r`. Additionally, the intermediate result is truncated with a frequency `trunc_freq`.

In [10]:
Y_all = [teneva.rand([5]*10, 2) for _ in range(10)]     # 10 random TT-tensors with TT-rank 2
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
teneva.show(Y)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5 20 20 20 20 20 20 20  5  1 



This function also supports float arguments:

In [11]:
Y_all = [
    42.,
    teneva.rand([5]*10, 2),
    33.,
    teneva.rand([5]*10, 4)
]
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
teneva.show(Y)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  7  7  7  7  7  7  7  5  1 



If all arguments are numbers, then function returns the sum of numbers:

In [12]:
Y_all = [10., 20., 2., 10.]
Y = teneva.add_many(Y_all, e=1.E-4, r=50, trunc_freq=2)
print(Y)

42.0


## Function `copy`

Return a copy of the given TT-tensor.

In [14]:
Y = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Z = teneva.copy(Y)         # The copy of Y
print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

0.5136144642310962
0.5136144642310962


In [15]:
Z[2][1, 2, 0] = 42.

print(Y[2][1, 2, 0])
print(Z[2][1, 2, 0])

0.5136144642310962
42.0


It also supports numbers for convenience:

In [16]:
teneva.copy(42.)

42.0

## Function `erank`

Compute the effective TT-rank of the given TT-tensor, i.e. constant rank of TT-tensor, which would have the same number of parameters as the given TT-tensor.

In [17]:
Y = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
teneva.erank(Y)            # The effective TT-rank

2.0

## Function `full`

In [ ]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

For a given TT-tensor (list of TT-cores), calculates the tensor in full format (this function can only be used for relatively small tensors).

In [18]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)           # Print the TT-tensor
Y2 = teneva.full(Y1)      # Compute full tensor from the TT-tensor
abs(np.max(Y2-Y0))        # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



2.0872192862952943e-14

## Function `get`

Compute the element with multiindex $k$ from the TT-tensor $Y$. See also a function `tensor.getter` that performs the same operation, but with an acceleration.

In [19]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
teneva.show(Y1)           # Print the TT-tensor
k = [1, 2, 3, 4, 5]       # Select some tensor element
y1 = teneva.get(Y1, k)    # Compute the element of the TT-tensor
y0 = Y0[tuple(k)]         # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

   10  10  10  10  10 
  / \ / \ / \ / \ / \ 
 1   10 100 100  10  1  



2.886579864025407e-15

## Function `getter`

Build fast (accelerated by numba) function that computes the element with multiindex $k$ for the TT-tensor $Y$. See also a function `tensor.get` for more details.

In [20]:
n = [10] * 5              # Shape of the tensor      
Y0 = np.random.randn(*n)  # Create 5-dim random numpy tensor
Y1 = teneva.svd(Y0)       # Compute TT-tensor from Y0 by TT-SVD
get = teneva.getter(Y1)   # Build (compile) function to compute the element of the TT-tensor
k = (1, 2, 3, 4, 5)       # Select some tensor element
y1 = get(k)               # Compute the element of the TT-tensor
y0 = Y0[k]                # Compute the same element of the original tensor
abs(np.max(y1-y0))        # Compare original tensor and reconstructed tensor

9.992007221626409e-16

## Function `mean`

Compute mean value of the TT-tensor.

In [24]:
Y = teneva.rand([5]*10, 2)   # 10-dim random TT-tensor with TT-rank 2
m = teneva.mean(Y)           # The mean value

In [25]:
Y_full = teneva.full(Y)      # Compute tensor in the full format to check the result
m_full = np.mean(Y_full)     # The mean value for the numpy array
e = abs(m - m_full)          # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')

Error     : 8.67e-19


The probability of tensor inputs my be also set:

In [26]:
n = [5]*10                   # Shape of the tensor
Y = teneva.rand(n, 2)        # 10-dim random TT-tensor with TT-rank 2
P = [np.zeros(k) for k in n] # The "probability"
teneva.mean(Y, P)            # The mean value

0.0

## Function `mul`

Compute element wise product $Y = Y_1 + Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [27]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [28]:
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 3 = 6)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  6  6  6  6  6  6  6  6  6  1 



In [29]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full * Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                     #

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 3.88e-16


This function also supports float argument:

In [30]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 1 = 2)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  2  2  2  2  2  2  2  2  2  1 



In [31]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 x 1 = 2)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  2  2  2  2  2  2  2  2  2  1 



In [32]:
Y1 = 21.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.mul(Y1, Y2)      # Compute the product of Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `mul_scalar`

Compute scalar product for Y1 and Y2 in the TT-format.

In [33]:
Y1 = teneva.rand([5]*10, 2)           # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3)           # 10-dim random TT-tensor with TT-rank 3

In [34]:
v = teneva.mul_scalar(Y1, Y2)         # Compute the product of Y1 and Y2
print(v)                              # Print the resulting value

-2116553.9954934935


In [35]:
Y1_full = teneva.full(Y1)             # Compute tensors in the full format
Y2_full = teneva.full(Y2)             # to check the result

v_full = np.sum(Y1_full * Y2_full)
print(v_full)                         # Print the resulting value from full tensor

e = abs((v - v_full)/v_full)          # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')       # Rel. error

-2116553.9954934916
Error     : 8.80e-16


## Function `norm`

Compute Frobenius norm of the given TT-tensor.

In [36]:
Y = teneva.rand([5]*10, 2)            # 10-dim random TT-tensor with TT-rank 2

In [37]:
v = teneva.norm(Y)                    # Compute the Frobenius norm
print(v)                              # Print the resulting value

11365.20404903054


In [38]:
Y_full = teneva.full(Y)               # Compute tensor in the full format to check the result

v_full = np.linalg.norm(Y_full)
print(v_full)                         # Print the resulting value from full tensor

e = abs((v - v_full)/v_full)          # Compute error for TT-tensor vs full tensor 
print(f'Error     : {e:-8.2e}')       # Rel. error

11365.204049030537
Error     : 3.20e-16


## Function `rand`

Construct random TT-tensor.

In [39]:
n = [12, 13, 14, 15, 16]    # Shape of the tensor
r = [1, 2, 3, 4, 5, 1]      # TT-ranks for TT-tensor
Y = teneva.rand(n, r)       # Build random TT-tensor
teneva.show(Y)              # Print the resulting TT-tensor

 12 13 14 15 16 
 / \/ \/ \/ \/ \
 1  2  3  4  5  1 



If all inner TT-ranks are equal, we may pass it as a number:

In [40]:
n = [12, 13, 14, 15, 16]    # Shape of the tensor
r = 5                       # TT-ranks for TT-tensor
Y = teneva.rand(n, r)       # Build random TT-tensor
teneva.show(Y)              # Print the resulting TT-tensor

 12 13 14 15 16 
 / \/ \/ \/ \/ \
 1  5  5  5  5  1 



## Function `ranks`

Function get the TT-ranks of the given TT-tensor.

In [41]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.ranks(Y)                        # TT-ranks of the TT-tensor

array([1, 2, 2, 2, 2, 1])

## Function `shape`

Function get the shape of the given TT-tensor.

In [42]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.shape(Y)                        # Shape of the TT-tensor

array([10, 12,  8,  8, 30])

## Function `show`

Displays mode sizes and TT-ranks of the given TT-tensor Y in a compact and clear form.

In [43]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.show(Y)                         # Print the resulting TT-tensor

 10 12  8  8 30 
 / \/ \/ \/ \/ \
 1  2  2  2  2  1 



## Function `size`

Function get the size (number of parameters) of the given TT-tensor.

In [44]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.size(Y)                         # Size of the TT-tensor

192

## Function `sub`

Compute element wise difference $Y = Y_1 - Y_2$ for the given TT-tensors $Y_1$ and $Y_2$ presented as lists of TT-cores.

In [45]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = teneva.rand([5]*10, 3) # 10-dim random TT-tensor with TT-rank 3

In [46]:
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 3 = 5)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  5  5  5  5  5  5  5  5  5  1 



In [47]:
Y1_full = teneva.full(Y1)                       # Compute tensors in the full format
Y2_full = teneva.full(Y2)                       # to check the result
Y_full = teneva.full(Y)

Z_full = Y1_full - Y2_full

e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                     

print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Error     : 1.01e-16


This function also supports float argument:

In [48]:
Y1 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y2 = 42.                    # Just a number
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [49]:
Y1 = 42.                    # Just a number
Y2 = teneva.rand([5]*10, 2) # 10-dim random TT-tensor with TT-rank 2
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
teneva.show(Y)              # Print the resulting TT-tensor (note that it has TT-rank 2 + 1 = 3)

  5  5  5  5  5  5  5  5  5  5 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  3  3  3  3  1 



In [50]:
Y1 = 44.                    # Just a number
Y2 = 2                      # Just a number
Y = teneva.sub(Y1, Y2)      # Compute the difference between Y1 and Y2
print(Y)                    # The result is a number

42.0


## Function `sum`

Compute sum of all tensor elements.

In [51]:
Y = teneva.rand([10, 12, 8, 8, 30], 2) # 5-dim random TT-tensor with TT-rank 2
teneva.sum(Y)                          # Sum of the TT-tensor elements

-57.64236038244924

In [52]:
Z = teneva.full(Y)                     # Compute tensors in the full format to check the result
np.sum(Z)

-57.64236038244928

---